In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda, num_samples=config.num_train_size)
# build dimension reduction model
hnne = get_hnne_model(X, y)
# get loader
loader = get_loader(X, y, hnne=hnne)
# get val loader
val_loader = get_val_loader(robot=panda, hnne=hnne)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from ./weights/panda/hnne.pickle
Model load successfully from ./weights/panda/nsf.pth


In [3]:
mu = torch.randn(size=(7,))
mu

tensor([0.1923, 0.6574, 0.9909, 0.6380, 1.6717, 1.3232, 0.1037])

In [4]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0)
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=val_loader, 
                      model=iflow, 
                      num_data=1, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,1.000000,1.00
mean,0.003971,0.26
std,NaN,NaN
min,0.003971,0.26
25%,0.003971,0.26
50%,0.003971,0.26
75%,0.003971,0.26
max,0.003971,0.26


In [5]:
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=val_loader, 
                      model=flow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.004205,0.054300
std,0.003059,0.011215
min,0.002082,0.050000
25%,0.002687,0.050000
50%,0.003515,0.050000
75%,0.004399,0.060000
max,0.024869,0.150000


In [6]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=val_loader, 
                      model=nflow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.003119,0.052000
std,0.002933,0.004264
min,0.001524,0.050000
25%,0.002007,0.050000
50%,0.002378,0.050000
75%,0.002927,0.050000
max,0.021238,0.070000


In [ ]:
traj_dir = sample_ee_traj(robot=panda, load_time='')

In [ ]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=nflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

In [ ]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=iflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)